## sqlite3

In [ ]:
import sqlite3

db = sqlite3.connect('./orm.db')
cursor = db.cursor()
cursor.execute("select * from book")

#参数化查询 防止注入
sql = "insert into book(title,content) VALUES(?,?)"
cursor.execute(sql,('Alice','alice another book'))

## sqlmodel

In [ ]:
from sqlmodel import SQLModel, Field, create_engine,Session,Relationship, select

engine = create_engine('sqlite:///orm.db')

class Author(SQLModel,table=True):
    id: int = Field(default=None, primary_key=True)
    name: str=Field(max_length=50)
    email:str=Field(max_length=50)
    books: list["Book"]= Relationship(back_populates="author")

class Book(SQLModel, table=True):
    id: int =Field(default=None, primary_key=True)
    title:str=Field(max_length=100)
    content:str
    author_id: int = Field(foreign_key="author.id")

    author: Author = Relationship(back_populates="books")

SQLModel.metadata.create_all(engine)
 
with Session(engine)as session:
    author1 = Author(name='Alice',email='alice@example.com')
    author2 = Author(name='Bob',email='bob@example.com')
    book1 = Book(title='Alice s First Book', content='This is the content of Alice s first book,', author=author1)
    book2 = Book(title='Alice s Second Book', content='This is the content of Alice s second book.', author=author1)
    book3 = Book(title='Bob s First Book', content='This is the content of Bob s first book.', author=author2)

session.add_all([author1, author2, book1, book2, book3])
session.commit()


In [ ]:
#全表查询
with Session(engine)as session:
    statement =select(Book)
    results =session.exec(statement).all()

for book in results:
    print(book)

title='Alice s First Book' content='This is the content of Alice s first book,' author_id=1 id=1
title='Alice s Second Book' content='This is the content of Alice s second book.' author_id=1 id=2
title='Bob s First Book' content='This is the content of Bob s first book.' author_id=2 id=3


In [ ]:
#where子句查询
with Session(engine)as session:
    statement =select(Book).where  (Book.title=='Alice s Second Book')
    results =session.exec(statement).all()

for book in results:
    print(book)

title='Alice s First Book' content='This is the content of Alice s first book,' author_id=1 id=1


In [ ]:
#join连结查询
with Session(engine)as session:
    statement =select(Book,Author).join(Author)
    books_with_authors =session.exec(statement).all()
for book, author in books_with_authors:
    print(f"Book: {book.title}, Author:{author.name}")

Book: Alice s Updated First Book, Author:Alice
Book: Alice s Second Book, Author:Alice
Book: Bob s First Book, Author:Bob


In [ ]:
#update 修改
with Session(engine)as session:
    statement =select(Book).where(Book.title == 'Alice s First Book')
    book_to_delete = session.exec(statement).first()

if book_to_delete:
    book_to_delete.title ='Alice s Updated First Book'
    session.add(book_to_delete)
    session.commit()
    session.refresh(book_to_delete)
    print(f"Updated book: {book_to_delete.title}")

Updated book: Alice s Updated First Book


In [ ]:
#delete删除
with Session(engine) as session:
    statement =select(Book).where(Book.title == 'Alice s Updated First Book')
    book_to_delete = session.exec(statement).first()

if book_to_delete:  
    session.add(book_to_delete)
    session.commit() 